In [ ]:
import websocket
import config
import ccxt
import json
import pandas as pd

In [ ]:
exchange =  ccxt.binance({
    'apiKey' : config.API_KEY,
    'secret' : config.API_SECRET
})

In [ ]:
balanceSpot = exchange.fetch_balance({
    'type' : 'spot'
})
balanceMargin = exchange.fetch_balance({
    'type' : 'margin'
})


In [ ]:
symbols = set()
for bal in balanceSpot['info']['balances']:
    if float(bal['free']) > 0:
        symbols.add(bal['asset'])

In [ ]:
for val in balanceMargin['info']['userAssets']:
    if float(val['netAsset']) > 0:
        symbols.add(val['asset'])

In [ ]:
symbols.remove('USDT')
print(symbols)

In [ ]:
since = exchange.milliseconds () - 30 * 86400000  # -1 day from now
orders = []
for symbol in symbols:
    SymbolinUSDT = f'{symbol}/USDT'
    orders += exchange.fetch_closed_orders(SymbolinUSDT,since, 10 ,{'type' : 'margin'})

In [144]:
buyOrders = {}
for val in orders:
    if val['info']['side'] == 'BUY':
        symbol = val['info']['symbol'].lower()
        quantity = val['info']['executedQty']
        totalMoneyinUSDT = val['info']['cummulativeQuoteQty']
        buyOrders[symbol] = {'totalamountinUSDT': totalMoneyinUSDT, 'quantity' : quantity}
        #print(val)
print(buyOrders)

{'bttusdt': {'totalamountinUSDT': '46.143996', 'quantity': '5053'}, 'linkusdt': {'totalamountinUSDT': '199.96811', 'quantity': '5.015'}, 'eosusdt': {'totalamountinUSDT': '199.99596', 'quantity': '30.83'}, 'dotusdt': {'totalamountinUSDT': '450.95892', 'quantity': '12.45'}, 'adausdt': {'totalamountinUSDT': '209.95146', 'quantity': '154.9'}, 'ltcusdt': {'totalamountinUSDT': '629.4634875', 'quantity': '2.29941'}, 'neousdt': {'totalamountinUSDT': '199.912676', 'quantity': '2.066'}}


In [145]:
with open('buyOrders.json', 'r') as openfile:
    buyOrders = json.load(openfile)
symbolList = buyOrders.keys()
symbolurl = ""
for x in symbolList:
    symbolurl += f'{x.lower()}@kline_1m/'
symbolurl = symbolurl[:-1]
#print(symbolurl)
socket = f"wss://stream.binance.com:9443/stream?streams={symbolurl}"
#socket = f"wss://stream.binance.com:9443/stream?streams=adausdt@kline_1m"
#print(socket)
print(buyOrders)

{'bttusdt': {'totalamountinUSDT': '383.53', 'quantity': '49152'}, 'xrpusdt': {'totalamountinUSDT': '511.18', 'quantity': '312.6'}, 'maticusdt': {'totalamountinUSDT': '255.59', 'quantity': '334'}, 'vetusdt': {'totalamountinUSDT': '383.53', 'quantity': '1610'}, 'adausdt': {'totalamountinUSDT': '255.59', 'quantity': '160.6'}, 'etcusdt': {'totalamountinUSDT': '256', 'quantity': '1.97'}, 'wrxusdt': {'totalamountinUSDT': '180.19', 'quantity': '74.2'}, 'dotusdt': {'totalamountinUSDT': '383.38', 'quantity': '9.11'}, 'dogeusdt': {'totalamountinUSDT': '255.59', 'quantity': '514'}}


In [148]:
from IPython.display import clear_output
from tabulate import tabulate
data = {}
usdtToINR = 78.25
def on_message(ws, message):
    obj = json.loads(message)
    symbol, cur_price = obj['data']['s'].lower(), float(obj['data']['k']['c'])
    buy_price = float(buyOrders[symbol]['totalamountinUSDT'])
    new_price = cur_price * float(buyOrders[symbol]['quantity'])
    perChange = (new_price - buy_price) * 100 / buy_price

    new_priceINR = new_price * usdtToINR
    new_buypriceINR =  buy_price * usdtToINR
    new_curpriceINR = cur_price * usdtToINR
    data[symbol] = {
            'PercentageChange' : perChange,
            'Current_Wallet' : new_priceINR,
            'Buy_price' : new_buypriceINR, 
            'CurrentPrice' : new_curpriceINR, 
            'Profit/Loss' : new_buypriceINR - new_priceINR
        }
    clear_output(wait=True)
    df = pd.DataFrame(data)
    df = df.T
    df = df.sort_values(by='PercentageChange')
    print(tabulate(df, headers = 'keys', tablefmt = 'psql'))
    print("{0} / {1}".format(int(df['Current_Wallet'].sum(axis=0)), int(df['Buy_price'].sum(axis=0))))

def on_close(ws):
    print("### closed ###")

def on_error(ws, error):
    print(error)

ws = websocket.WebSocketApp(socket ,on_message = on_message, on_close = on_close, on_error=on_error)

In [149]:
ws.run_forever()

+-----------+--------------------+------------------+-------------+----------------+
|           |   PercentageChange |   Current_Wallet |   Buy_price |   CurrentPrice |
|-----------+--------------------+------------------+-------------+----------------|
| etcusdt   |          -8.60123  |          18309   |     20032   |     9293.91    |
| xrpusdt   |          -5.99618  |          37601.4 |     39999.8 |      120.286   |
| dotusdt   |          -5.31664  |          28404.5 |     29999.5 |     3117.95    |
| bttusdt   |          -5.22288  |          28443.8 |     30011.2 |        0.57869 |
| vetusdt   |          -2.09783  |          29381.6 |     30011.2 |       18.2495  |
| wrxusdt   |          -0.792142 |          13988.2 |     14099.9 |      188.52    |
| dogeusdt  |          12.0648   |          22412.9 |     19999.9 |       43.6048  |
| adausdt   |          13.4863   |          22697.2 |     19999.9 |      141.327   |
| maticusdt |          33.732    |          26746.3 |     19999.9

False

In [ ]:
import json
  
with open("buyOrders.json", "w") as outfile:
    json.dump(buyOrders, outfile)

In [142]:
df = pd.DataFrame(data)
df = df.T
df = df.sort_values(by='PercentageChange')
print(tabulate(df, headers = 'keys', tablefmt = 'psql'))
print("{0} / {1}".format(int(df['Current_Wallet'].sum(axis=0)), int(df['Buy_price'].sum(axis=0))))

+-----------+--------------------+------------------+-------------+----------------+
|           |   PercentageChange |   Current_Wallet |   Buy_price |   CurrentPrice |
|-----------+--------------------+------------------+-------------+----------------|
| etcusdt   |           -6.81438 |          18666.9 |     20032   |    9475.61     |
| xrpusdt   |           -5.88611 |          37645.4 |     39999.8 |     120.427    |
| dotusdt   |           -5.1907  |          28442.3 |     29999.5 |    3122.1      |
| bttusdt   |           -3.41843 |          28985.3 |     30011.2 |       0.589708 |
| vetusdt   |           -1.81237 |          29467.3 |     30011.2 |      18.3027   |
| wrxusdt   |           -1.0351  |          13953.9 |     14099.9 |     188.058    |
| dogeusdt  |           12.3303  |          22466   |     19999.9 |      43.7081   |
| adausdt   |           13.1973  |          22639.4 |     19999.9 |     140.967    |
| maticusdt |           34.9591  |          26991.7 |     19999.9